In [1]:
# This code writes the product data table to a TXT file.

product_data = [
    ["Metric", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Annual demand (units)", "10,000", "15,000", "25,000", "2,000", "1,500", "95,000"],
    ["Order cost ($)", "50", "50", "50", "50", "50", "150"],
    ["Price/unit load ($)", "500", "650", "350", "250", "225", "150"],
    ["Space required (m²)", "10", "15", "25", "10", "12", "13"],
    ["Reserve dwell percentage (%)", "0", "0", "20%", "0", "0", "100%"],
    ["Yearly carrying cost rate (%)", "10%", "10%", "10%", "10%", "10%", "10%"]
]

with open("product_data.txt", "w") as f:
    for row in product_data:
        f.write("\t".join(row) + "\n")

In [7]:
# This code writes the flow/product data table to a TXT file.

flow_product_data = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "0.0707", "0.0203", "0.0267", "0.3354", "0.4083", "0.0726"],
    ["Flow 2 (R)", "0.0849", "0.2023", "0.0420", "0.5590", "0.6804", "0.0871"],
    ["Flow 3 (RF)", "0.1061", "0.2023", "0.0054", "1.0062", "1.2248", "0.1088"],
    ["Flow 4 (F)", "0.0778", "0.2023", "0.0481", "0.0671", "0.8165", "0.0798"]
]

with open("product_cost_data.txt", "w") as f:
    for row in flow_product_data:
        f.write("\t".join(row) + "\n")

In [8]:
# This code writes the new flow/product quantity table to a TXT file.

flow_product_quantity = [
    ["Flow/Product", "Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"],
    ["Flow 1 (CD)", "20", "15", "4", "5", "15", "20"],
    ["Flow 2 (R)", "5", "5", "20", "4", "25", "5"],
    ["Flow 3 (RF)", "10", "10", "1", "5", "45", "10"],
    ["Flow 4 (F)", "15", "10", "9", "1", "30", "15"]
]

with open("product_yearly_cost.txt", "w") as f:
    for row in flow_product_quantity:
        f.write("\t".join(row) + "\n")

In [4]:
# This code writes the functional area bounds table to a TXT file.

functional_area_bounds = [
    ["Functional Area", "Lower bound (m²)", "Upper bound (m²)"],
    ["Cross-docking", "0", "15,000"],
    ["Reserve", "35,000", "75,000"],
    ["Forward", "35,000", "75,000"]
]

with open("functional_area_bounds.txt", "w") as f:
    for row in functional_area_bounds:
        f.write("\t".join(row) + "\n")

In [5]:
# This code writes the levels of space available in each functional area to a TXT file.

functional_area_levels = [
    ["Functional Area", "#Levels"],
    ["Cross-docking", "1"],
    ["Reserve", "1"],
    ["Forward", "1"]
]

with open("functional_area_levels.txt", "w") as f:
    for row in functional_area_levels:
        f.write("\t".join(row) + "\n")

In [1]:
# Calculate holding cost per unit for each product and functional area
# Uses yearly carrying cost rate, price per product, and storage cost from product_yearly_cost.txt

import csv

carry_rate = [0.10, 0.10, 0.10, 0.10, 0.10, 0.10]  # yearly carrying cost rate for each product
price = [500, 650, 350, 250, 225, 150]             # price per unit load for each product
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]

# Read storage costs from product_yearly_cost.txt
storage = []
with open("product_yearly_cost.txt", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    next(reader)  # skip header
    for row in reader:
        storage.append([float(x) for x in row[1:]])

holding_cost = []  # [functional_area, product, holding_cost]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        h_pf = carry_rate[p] * price[p] + storage[f][p]
        holding_cost.append([area, prod, round(h_pf, 2)])

# Write results to file
with open("holding_cost_per_unit.txt", "w") as f:
    f.write("Functional Area\tProduct\tHolding Cost per Unit\n")
    for row in holding_cost:
        f.write("\t".join(map(str, row)) + "\n")

In [4]:
# Calculate EOQ for each product p assigned to flow f
# EOQ = sqrt((2 * D * S) / H)
# D = annual demand, S = order cost, H = holding cost per unit (from previous calculation)

import math

# Data for each product
annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D
order_cost = [50, 50, 50, 50, 50, 150]                   # S

# Read holding cost per unit from holding_cost_per_unit.txt
# We'll use the same order of functional_areas and products as before
holding_cost = []  # 2D list: [functional_area][product]
with open("holding_cost_per_unit.txt", "r") as f:
    next(f)  # skip header
    for i in range(4):  # 4 functional areas
        row = []
        for j in range(6):  # 6 products
            line = f.readline()
            h = float(line.strip().split('\t')[2])
            row.append(h)
        holding_cost.append(row)

# Calculate EOQ for each product in each functional area
eoq = []  # [functional_area, product, EOQ]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        D = annual_demand[p]
        S = order_cost[p]
        H = holding_cost[f][p]
        EOQ = math.sqrt((2 * D * S) / H) if H > 0 else 0
        eoq.append([area, prod, round(EOQ, 2)])

# Write EOQ results to file
with open("eoq_per_product_flow.txt", "w") as f:
    f.write("Functional Area\tProduct\tEOQ\n")
    for row in eoq:
        f.write("\t".join(map(str, row)) + "\n")

In [5]:
# Calculate average inventory position for each product and functional area as EOQ/2

import math

# Data for each product
annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D
order_cost = [50, 50, 50, 50, 50, 150]                   # S

# Read holding cost per unit from holding_cost_per_unit.txt
holding_cost = []  # 2D list: [functional_area][product]
with open("holding_cost_per_unit.txt", "r") as f:
    next(f)  # skip header
    for i in range(4):  # 4 functional areas
        row = []
        for j in range(6):  # 6 products
            line = f.readline()
            h = float(line.strip().split('\t')[2])
            row.append(h)
        holding_cost.append(row)

# Calculate EOQ and average inventory position for each product in each functional area
avg_inventory = []  # [functional_area, product, avg_inventory]
functional_areas = ["Cross-docking", "Reserve", "RF", "Forward"]
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

for f, area in enumerate(functional_areas):
    for p, prod in enumerate(products):
        D = annual_demand[p]
        S = order_cost[p]
        H = holding_cost[f][p]
        EOQ = math.sqrt((2 * D * S) / H) if H > 0 else 0
        avg_inv = EOQ / 2
        avg_inventory.append([area, prod, round(avg_inv, 2)])

# Write average inventory results to file
with open("avg_inventory_position.txt", "w") as f:
    f.write("Functional Area\tProduct\tAverage Inventory Position\n")
    for row in avg_inventory:
        f.write("\t".join(map(str, row)) + "\n")

In [6]:
# For Flow 3 (Reserve→Forward): allocation of average inventory to reserve & forward areas
# Let r = reserve_dwell[p] (fraction). For product p assigned to flow 3:
# Reserve area gets r * avg_inventory, Forward area gets (1 - r) * avg_inventory

reserve_dwell = [0, 0, 0.2, 0, 0, 1.0]  # as fractions for each product (from your table)
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]

# Read average inventory for Flow 3 (RF) from avg_inventory_position.txt
avg_inventory_flow3 = []
with open("avg_inventory_position.txt", "r") as f:
    next(f)  # skip header
    for _ in range(12):  # skip first 12 rows (3 flows x 6 products)
        next(f)
    for _ in range(6):  # next 6 rows are Flow 3 (RF)
        line = f.readline()
        avg_inv = float(line.strip().split('\t')[2])
        avg_inventory_flow3.append(avg_inv)

# Allocate average inventory to Reserve and Forward areas for Flow 3
allocation = []  # [Product, Reserve Alloc, Forward Alloc]
for p, prod in enumerate(products):
    r = reserve_dwell[p]
    avg_inv = avg_inventory_flow3[p]
    reserve_alloc = round(r * avg_inv, 2)
    forward_alloc = round((1 - r) * avg_inv, 2)
    allocation.append([prod, reserve_alloc, forward_alloc])

# Write allocation to file
with open("avg_inventory_allocation_flow3.txt", "w") as f:
    f.write("Product\tReserve Allocation\tForward Allocation\n")
    for row in allocation:
        f.write("\t".join(map(str, row)) + "\n")

In [8]:
# Optimization model for warehouse flows and costs

import csv
import math

# --- PARAMETERS (edit as needed) ---
products = ["Product 1", "Product 2", "Product 3", "Product 4", "Product 5", "Product 6"]
flows = ["Flow 1 (CD)", "Flow 2 (R)", "Flow 3 (RF)", "Flow 4 (F)"]
functional_areas = ["Cross-docking", "Reserve", "Forward"]

annual_demand = [10000, 15000, 25000, 2000, 1500, 95000]  # D[p]
order_cost = [50, 50, 50, 50, 50, 150]                   # Co[p]
handling_cost = [10, 12, 8, 7, 6, 5]                     # handling cost per unit load for each product (example)
space_per_unit = [10, 15, 25, 10, 12, 13]                # m² per unit
reserve_dwell = [0, 0, 0.2, 0, 0, 1.0]                   # for Flow 3 (RF)
B = [1, 1, 1, 1, 1, 1]                                   # break-bulk factor (user can change)

# --- READ EOQ, AvgInv, h_{p,f} ---
# Read EOQ
EOQ = [[0]*6 for _ in range(4)]
with open("eoq_per_product_flow.txt", "r") as f:
    next(f)
    for i in range(24):
        area, prod, eoq = f.readline().strip().split('\t')
        EOQ[i//6][i%6] = float(eoq)

# Read AvgInv
AvgInv = [[0]*6 for _ in range(4)]
with open("avg_inventory_position.txt", "r") as f:
    next(f)
    for i in range(24):
        area, prod, avg_inv = f.readline().strip().split('\t')
        AvgInv[i//6][i%6] = float(avg_inv)

# Read h_{p,f}
h_pf = [[0]*6 for _ in range(4)]
with open("holding_cost_per_unit.txt", "r") as f:
    next(f)
    for i in range(24):
        area, prod, h = f.readline().strip().split('\t')
        h_pf[i//6][i%6] = float(h)

# --- OPTIMIZATION VARIABLES (example: assign each product to a flow, here fixed for demo) ---
# Assignments: product_flow[p] = f (0=CD, 1=R, 2=RF, 3=F)
# For demo, assign as in your flow_product_quantity table (edit as needed)
product_flow = [0, 0, 2, 0, 2, 2]  # Example: Product 1,2,4 in CD; 3,5,6 in RF

# --- COSTS AND AREA USAGE ---
total_cost = 0
area_CD = 0
area_R = 0
area_F = 0

for p, prod in enumerate(products):
    f = product_flow[p]
    D = annual_demand[p]
    Co = order_cost[p]
    H = h_pf[f][p]
    eoq = EOQ[f][p]
    avg_inv = AvgInv[f][p]
    hand_cost = handling_cost[p]
    scost = avg_inv * H
    hcost = D * hand_cost
    ocost = (D / eoq) * Co if eoq > 0 else 0
    total = hcost + ocost + scost

    # Area usage by flow
    if f == 0:  # Flow 1 (CD)
        area_CD += avg_inv * space_per_unit[p]
    elif f == 1:  # Flow 2 (R)
        area_R += avg_inv * space_per_unit[p]
    elif f == 2:  # Flow 3 (RF)
        # Split avg_inv into reserve and forward
        r = reserve_dwell[p]
        avg_inv_reserve = r * avg_inv
        avg_inv_forward = (1 - r) * avg_inv
        area_R += avg_inv_reserve * space_per_unit[p]
        area_F += avg_inv_forward * B[p] * space_per_unit[p]
    elif f == 3:  # Flow 4 (F)
        area_F += avg_inv * space_per_unit[p]

    print(f"{prod} (Flow {f+1}): HC={hcost:.2f}, OC={ocost:.2f}, SC={scost:.2f}, Total={total:.2f}")

    total_cost += total

print(f"\nTotal Cost: {total_cost:.2f}")
print(f"Area Cross-dock: {area_CD:.2f} m²")
print(f"Area Reserve: {area_R:.2f} m²")
print(f"Area Forward: {area_F:.2f} m²")

Product 1 (Flow 1): HC=100000.00, OC=4183.40, SC=4183.20, Total=108366.60
Product 2 (Flow 1): HC=180000.00, OC=5477.25, SC=5477.60, Total=190954.85
Product 3 (Flow 3): HC=200000.00, OC=4743.47, SC=4743.36, Total=209486.83
Product 4 (Flow 1): HC=14000.00, OC=1224.74, SC=1224.60, Total=16449.34
Product 5 (Flow 3): HC=9000.00, OC=1591.01, SC=1590.97, Total=12181.98
Product 6 (Flow 3): HC=475000.00, OC=13346.32, SC=13346.25, Total=501692.57

Total Cost: 1039132.18
Area Cross-dock: 2032.85 m²
Area Reserve: 7598.85 m²
Area Forward: 2918.04 m²
